In [1]:
import os

os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import logging
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, LSTM, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.layers import RNN, LSTMCell
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from sklearn.model_selection import train_test_split

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger(__name__)

RAW_DATA_PATH = "../data/activity_data_raw.csv"
PLANNED_WORKOUTS_PATH = "../data/planned_workouts.csv"
TARGET_FEATURES = ["power_5s_avg"]
FEATURES = ["grade", "ascent_meters", "distance_meters", "atl_start", "ctl_start", "watt_kg", "temperature", "distance_diff", "grade_5s_avg", "grade_diff"]
ALL_FEATURES = TARGET_FEATURES + FEATURES
MODEL_PATH = "../models/LSTM_power.keras"
METRICS_PATH = "../data/model_metrics_power.csv"

def load_activity_data(csv_path):
    logger.info(f"Loading data from {csv_path}")
    df = pd.read_csv(csv_path)
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    logger.info("Data loaded successfully")
    return df

def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

def preprocess_data(df, drop_activities_with_workout=False):
    logger.info("Starting data preprocessing")
    df = df.sort_values(by=["activity_id", "timestamp"])
    df["altitude_diff"] = df["altitude"].diff()
    df["distance_diff"] = df["distance"].diff()
    df["ascent_meters"] = df["altitude_diff"].apply(lambda x: x if x > 0 else 0).cumsum()
    df["grade_diff"] = df["grade"].diff().fillna(0)
    df["seconds_from_start"] = (df["timestamp"] - df["timestamp"].iloc[0]).dt.total_seconds()
    df.rename(columns={"distance": "distance_meters"}, inplace=True)
    
    df.fillna(method='ffill', inplace=True)
    df.fillna(method='bfill', inplace=True)

    df["power_5s_avg"] = df["power"].rolling(window=5, min_periods=1).mean()

    planned_workouts = pd.read_csv(PLANNED_WORKOUTS_PATH)

    if drop_activities_with_workout:
        intersecting_activities = planned_workouts["paired_activity_id"].unique()
        logger.info(f"Dropping {len(intersecting_activities)} activities with planned workouts")
        df = df[~df["activity_id"].isin(intersecting_activities)]
        logger.info(f"Remaining activities: {df['activity_id'].nunique()}")

    rows_na = df.isna().sum().sum()
    df.dropna(inplace=True)
    logger.info(f"Dropped {rows_na} rows with NaN values")
    if df[ALL_FEATURES].isnull().values.any() or np.isinf(df[ALL_FEATURES]).values.any():
        logger.error("NaN or infinite values found in the data")
        raise ValueError("NaN or infinite values found in the data")

    logger.info("Data preprocessing completed")
    return df

def split_by_activities(df, test_size=0.2, val_size=0.1):
    activity_ids = df['activity_id'].unique()
    train_ids, test_ids = train_test_split(activity_ids, test_size=test_size, random_state=42)
    train_ids, val_ids = train_test_split(train_ids, test_size=val_size / (1 - test_size), random_state=42)
    
    train_df = df[df['activity_id'].isin(train_ids)]
    val_df = df[df['activity_id'].isin(val_ids)]
    test_df = df[df['activity_id'].isin(test_ids)]
    
    return train_df, val_df, test_df

def scale_data(train, val, test):
    train_scaled = train.copy()
    val_scaled = val.copy()
    test_scaled = test.copy()
    
    features = FEATURES + TARGET_FEATURES
    
    scaler = StandardScaler()
    scaler.fit(train[features])
    
    train_scaled[features] = scaler.transform(train[features])
    val_scaled[features] = scaler.transform(val[features])
    test_scaled[features] = scaler.transform(test[features])
    
    return train_scaled, val_scaled, test_scaled

def augment_data(seq, aug_methods):
    method = np.random.choice(aug_methods)
    if method == 'noise':
        noise = np.random.normal(0, 0.01, seq.shape)
        return seq + noise
    elif method == 'scaling':
        factor = np.random.uniform(0.9, 1.1)
        return seq * factor
    elif method == 'shifting':
        shift = np.random.randint(-3, 3)
        return np.roll(seq, shift, axis=0)
    elif method == 'time_warping':
        time_steps = np.arange(seq.shape[0])
        warp = np.interp(time_steps, time_steps, seq)
        return warp
    return seq

def create_sequences(data, sequence_length, target_columns, augmentation=False):
    num_records = len(data)
    num_features = len(FEATURES)
    num_targets = len(target_columns)

    sequences = np.zeros((num_records - sequence_length, sequence_length, num_features), dtype=np.float32)
    targets = np.zeros((num_records - sequence_length, num_targets), dtype=np.float32)

    feature_data = data[FEATURES].values
    target_data = data[target_columns].values

    for i in range(num_records - sequence_length):
        seq = feature_data[i:i + sequence_length]
        if augmentation:
            seq = augment_data(seq, ['noise', 'scaling', 'shifting'])
        sequences[i] = seq
        targets[i] = target_data[i + sequence_length]

    return sequences, targets

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

from tensorflow.keras.regularizers import l2

def build_model(
        learning_rate=0.001,
        cnn_filters=[],
        dense_units=[],
        lstm_units=[1],
        sequence_length=60,
        dropout_rate_dense=0, 
        dropout_rate_lstm=0,
        dropout_rate_cnn=0,
        add_batch_norm=False,
        l2_reg=0.01,
):
    model = Sequential()

    # CNN LAYERS
    if cnn_filters:
        for i, num_filters in enumerate(cnn_filters):
            if i == 0:
                model.add(tf.keras.layers.Conv1D(filters=num_filters, kernel_size=3, activation='relu', input_shape=(sequence_length, len(FEATURES))))
            else:
                model.add(tf.keras.layers.Conv1D(filters=num_filters, kernel_size=3, activation='relu'))
            model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
            model.add(Dropout(dropout_rate_cnn))
        if add_batch_norm:
            model.add(BatchNormalization())

    
    # LSTM LAYERS
    for i, num_units in enumerate(lstm_units):
        return_seq = True if i < len(lstm_units) - 1 else False
        model.add(RNN(LSTMCell(num_units, activation='tanh', kernel_regularizer=l2(l2_reg)), return_sequences=return_seq))
        if add_batch_norm:
            model.add(BatchNormalization())
        model.add(Dropout(dropout_rate_lstm))

    # DENSE LAYERS
    if dense_units:
        for i, num_units in enumerate(dense_units):
            model.add(Dense(num_units, activation='relu', kernel_regularizer=l2(l2_reg)))
            if add_batch_norm:
                model.add(BatchNormalization())
            model.add(Dropout(dropout_rate_dense))

    model.add(Dense(len(TARGET_FEATURES)))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss="mse")
    
    logger.info("LSTM model built and compiled")
    return model

def get_model_metrics(model, sequences, targets):
    logger.info("Getting model metrics")
    predictions = model.predict(sequences)
    metrics = {}
    for i, target in enumerate(TARGET_FEATURES):
        metric = tf.keras.metrics.mean_squared_error(targets[:, i], predictions[:, i]).numpy()
        metrics[target] = metric
    logger.info(f"Model metrics: {metrics}")
    return metrics

def plot_history(history):
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.legend()
    plt.show()

def predict_first_n_points(df, model, sequence_length, n_points=100, seed=42):
    activity_id = df['activity_id'].sample(1, random_state=seed).values[0]
    activity_data = df[df['activity_id'] == activity_id]
    
    if len(activity_data) < sequence_length + n_points:
        print(f"La actividad {activity_id} no tiene suficientes puntos para la predicción. Se requieren al menos {sequence_length + n_points} puntos, pero la actividad tiene {len(activity_data)} puntos.")
        return

    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(activity_data[FEATURES])
    
    sequences = []
    for i in range(n_points):
        seq = scaled_features[i:i + sequence_length]
        sequences.append(seq)
    
    sequences = np.array(sequences)
    predictions = model.predict(sequences)
    
    power_scaler = StandardScaler()
    power_scaler.fit(activity_data[TARGET_FEATURES])
    predictions = power_scaler.inverse_transform(predictions)
    
    actual_power = activity_data[TARGET_FEATURES].iloc[sequence_length:sequence_length + n_points].values
    
    fig, ax1 = plt.subplots(figsize=(14, 7))

    ax1.set_xlabel('Timestep')
    ax1.set_ylabel(TARGET_FEATURES[0])
    ax1.plot(range(n_points), actual_power, label='Actual %s' % TARGET_FEATURES[0], color='tab:blue')
    ax1.plot(range(n_points), predictions, label='Predicted Power', color='tab:red')
    ax1.tick_params(axis='y')

    fig.tight_layout()
    fig.legend(loc='upper left', bbox_to_anchor=(0.1, 0.9))
    plt.title(f'Activity ID: {activity_id}')
    plt.show()

In [2]:
LR_INIT = 0.001
SEQ_LENGTH = 30
BATCH_SIZE = 1024
EPOCHS = 100
augment_data_flag = True

# LSTM
LSTM_UNITS = [128, 64]
DROPOUT_RATE_LSTM = 0.2

# DENSE
DENSE_UNITS = [32, 16]
DROPOUT_RATE_DENSE = 0.2

# CNN
CNN_FILTERS = []
DROPOUT_RATE_CNN = 0.2

# Other
ADD_BATCH_NORM = True
PATIENCE = 10
L2_REG = 0.02

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

df = load_activity_data(RAW_DATA_PATH)
df = preprocess_data(df, drop_activities_with_workout=True)

train_df, val_df, test_df = split_by_activities(df)

train_scaled, val_scaled, test_scaled = scale_data(train_df, val_df, test_df)

train_sequences, train_targets = create_sequences(train_scaled, SEQ_LENGTH, TARGET_FEATURES, augmentation=augment_data_flag)
val_sequences, val_targets = create_sequences(val_scaled, SEQ_LENGTH, TARGET_FEATURES, augmentation=False)
test_sequences, test_targets = create_sequences(test_scaled, SEQ_LENGTH, TARGET_FEATURES, augmentation=False)

lr_scheduler = LearningRateScheduler(scheduler)
early_stopping = EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=True)

model = build_model(
    add_batch_norm=ADD_BATCH_NORM,
    learning_rate=LR_INIT,
    lstm_units=LSTM_UNITS,
    dense_units=DENSE_UNITS,
    cnn_filters=CNN_FILTERS,
    sequence_length=SEQ_LENGTH,
    dropout_rate_lstm=DROPOUT_RATE_LSTM,
    dropout_rate_dense=DROPOUT_RATE_DENSE,
    dropout_rate_cnn=DROPOUT_RATE_CNN,
    l2_reg=L2_REG
)

model.build(input_shape=(None, SEQ_LENGTH, len(FEATURES)))


# Print the number of sequences and targets that will be used for training
logger.info(f"Training sequences shape: {train_sequences.shape}")
logger.info(f"Training targets shape: {train_targets.shape}")

2024-06-08 01:00:03,773 - Loading data from ../data/activity_data_raw.csv
2024-06-08 01:00:04,418 - Data loaded successfully
2024-06-08 01:00:04,419 - Starting data preprocessing
C:\Users\Fco\AppData\Local\Temp\ipykernel_23904\4070497308.py:54: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
C:\Users\Fco\AppData\Local\Temp\ipykernel_23904\4070497308.py:55: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)
2024-06-08 01:00:04,616 - Dropping 39 activities with planned workouts
2024-06-08 01:00:04,630 - Remaining activities: 16
2024-06-08 01:00:04,654 - Dropped 0 rows with NaN values
2024-06-08 01:00:04,660 - Data preprocessing completed
2024-06-08 01:00:05,900 - LSTM model built and compiled
2024-06-08 01:00:06,231 - Training sequence

In [3]:
import optuna
from optuna_integration import TFKerasPruningCallback
import gc
from tensorflow.keras.callbacks import ModelCheckpoint
import pandas as pd
import subprocess
import time

# Función de construcción del modelo
def build_model(
        learning_rate=0.001,
        cnn_filters=(),
        dense_units=(),
        lstm_units=(),
        sequence_length=60,
        dropout_rate_dense=0, 
        dropout_rate_lstm=0,
        dropout_rate_cnn=0,
        add_batch_norm=False,
        l2_reg=0.01,
):
    model = Sequential()

    # CNN LAYERS
    if cnn_filters:
        for i, num_filters in enumerate(cnn_filters):
            if i == 0:
                model.add(tf.keras.layers.Conv1D(filters=num_filters, kernel_size=3, activation='relu', input_shape=(sequence_length, len(FEATURES))))
            else:
                model.add(tf.keras.layers.Conv1D(filters=num_filters, kernel_size=3, activation='relu'))
            model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
            model.add(Dropout(dropout_rate_cnn))
        if add_batch_norm:
            model.add(BatchNormalization())

    # LSTM LAYERS
    for i, num_units in enumerate(lstm_units):
        return_seq = True if i < len(lstm_units) - 1 else False
        model.add(RNN(LSTMCell(num_units, activation='tanh', kernel_regularizer=l2(l2_reg)), return_sequences=return_seq))
        if add_batch_norm:
            model.add(BatchNormalization())
        model.add(Dropout(dropout_rate_lstm))

    # DENSE LAYERS
    if dense_units:
        for i, num_units in enumerate(dense_units):
            model.add(Dense(num_units, activation='relu', kernel_regularizer=l2(l2_reg)))
            if add_batch_norm:
                model.add(BatchNormalization())
            model.add(Dropout(dropout_rate_dense))

    model.add(Dense(len(TARGET_FEATURES)))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss="mse")
    
    logger.info("LSTM model built and compiled")
    return model

def scheduler(epoch, lr):
    return lr * tf.math.exp(-0.1) if epoch >= 10 else lr

def objective(trial):
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    lstm_units = trial.suggest_categorical('lstm_units', 
                                           ((128,), (128, 128), (128, 64), (64, 32), (64, 64), (32, 32),
                                            (50, 50, 50), (100, 100, 100), (128, 64, 32), (64, 32, 16)))
    dense_units = trial.suggest_categorical('dense_units', 
                                            ((32, 16), (64, 32), (128, 64), (64, 64), (32, 32),
                                             (128, 64, 32), (64, 32, 16), (128, 64, 32)))
    dropout_rate_lstm = trial.suggest_float('dropout_rate_lstm', 0.0, 0.5)
    dropout_rate_dense = trial.suggest_float('dropout_rate_dense', 0.0, 0.5)
    dropout_rate_cnn = trial.suggest_float('dropout_rate_cnn', 0.0, 0.5)
    l2_reg = trial.suggest_float('l2_reg', 0.0, 0.1)
    cnn_filters = trial.suggest_categorical('cnn_filters', ((), (32, 64), (64, 128), (128, 64), (64, 32)))

    model = build_model(
        learning_rate=learning_rate,
        cnn_filters=cnn_filters,
        lstm_units=lstm_units,
        dense_units=dense_units,
        dropout_rate_lstm=dropout_rate_lstm,
        dropout_rate_dense=dropout_rate_dense,
        dropout_rate_cnn=dropout_rate_cnn,
        l2_reg=l2_reg,
        sequence_length=SEQ_LENGTH
    )
    
    batch_size = BATCH_SIZE
    checkpoint_callback = ModelCheckpoint(filepath=f'checkpoint_trial_{trial.number}.h5', save_weights_only=True, save_best_only=True, monitor='val_loss', verbose=1)
    
    history = model.fit(train_sequences, train_targets, batch_size=batch_size, epochs=EPOCHS, validation_data=(val_sequences, val_targets), callbacks=[
        TFKerasPruningCallback(trial, 'val_loss'),
        LearningRateScheduler(scheduler),
        EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=True),
        checkpoint_callback
    ], verbose=0)
    
    val_loss = min(history.history['val_loss'])
    del model
    tf.keras.backend.clear_session()
    gc.collect()
    
    return val_loss

def run_dashboard(storage_name):
    subprocess.Popen(["optuna-dashboard", storage_name])

study_name = "example_study"  
storage_name = "sqlite:///example_study.db"  

study = optuna.create_study(study_name=study_name, direction='minimize', storage=storage_name, load_if_exists=True)

try:
    study.optimize(objective, n_trials=50)  
except Exception as e:
    print(f"Optimization failed with exception: {e}")

best_trial = study.best_trial
results = {
    'learning_rate': best_trial.params['learning_rate'],
    'cnn_filters': best_trial.params['cnn_filters'],
    'lstm_units': best_trial.params['lstm_units'],
    'dense_units': best_trial.params['dense_units'],
    'dropout_rate_lstm': best_trial.params['dropout_rate_lstm'],
    'dropout_rate_dense': best_trial.params['dropout_rate_dense'],
    'dropout_rate_cnn': best_trial.params['dropout_rate_cnn'],
    'l2_reg': best_trial.params['l2_reg'],
    'val_loss': best_trial.value
}

results_df = pd.DataFrame([results])
results_df.to_csv(METRICS_PATH, index=False)

c:\Users\Fco\miniconda3\envs\tfdml_plugin\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-06-08 01:00:06,709] A new study created in RDB with name: example_study
c:\Users\Fco\miniconda3\envs\tfdml_plugin\lib\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (128,) which is of type tuple.
  warnings.warn(message)
c:\Users\Fco\miniconda3\envs\tfdml_plugin\lib\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (128, 128) which is of type tuple.
  warnings.warn(message)
c:\Users\Fco\miniconda3\envs\tfdml_plugin\lib\site-packages\optuna\distributions.


Epoch 1: val_loss improved from inf to 1.10174, saving model to checkpoint_trial_0.h5


c:\Users\Fco\miniconda3\envs\tfdml_plugin\lib\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [128] which is of type list.
  warnings.warn(message)
c:\Users\Fco\miniconda3\envs\tfdml_plugin\lib\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [128, 128] which is of type list.
  warnings.warn(message)
c:\Users\Fco\miniconda3\envs\tfdml_plugin\lib\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [128, 64] which is of type list.
  warnings.warn(message)
c:\Users\Fco\miniconda3\envs\tfdml_plugin\lib\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution s


Epoch 2: val_loss improved from 1.10174 to 0.87540, saving model to checkpoint_trial_0.h5

Epoch 3: val_loss improved from 0.87540 to 0.49779, saving model to checkpoint_trial_0.h5

Epoch 4: val_loss did not improve from 0.49779

Epoch 5: val_loss did not improve from 0.49779

Epoch 6: val_loss did not improve from 0.49779

Epoch 7: val_loss did not improve from 0.49779

Epoch 8: val_loss did not improve from 0.49779

Epoch 9: val_loss did not improve from 0.49779

Epoch 10: val_loss did not improve from 0.49779

Epoch 11: val_loss did not improve from 0.49779

Epoch 12: val_loss improved from 0.49779 to 0.47216, saving model to checkpoint_trial_0.h5

Epoch 13: val_loss did not improve from 0.47216

Epoch 14: val_loss did not improve from 0.47216

Epoch 15: val_loss did not improve from 0.47216

Epoch 16: val_loss did not improve from 0.47216

Epoch 17: val_loss did not improve from 0.47216

Epoch 18: val_loss did not improve from 0.47216

Epoch 19: val_loss did not improve from 0.472

[I 2024-06-08 01:01:22,800] Trial 0 finished with value: 0.4721550941467285 and parameters: {'learning_rate': 0.008250259191241739, 'lstm_units': [100, 100, 100], 'dense_units': [128, 64, 32], 'dropout_rate_lstm': 0.4912801438488539, 'dropout_rate_dense': 0.22085029232914855, 'dropout_rate_cnn': 0.04177913064883143, 'l2_reg': 0.06914926684234356, 'cnn_filters': [128, 64]}. Best is trial 0 with value: 0.4721550941467285.
2024-06-08 01:01:23,054 - LSTM model built and compiled


...

[I 2024-06-08 02:22:08,434] Trial 46 pruned. Trial was pruned at epoch 1.
2024-06-08 02:22:08,668 - LSTM model built and compiled



Epoch 1: val_loss improved from inf to 0.44703, saving model to checkpoint_trial_47.h5


[I 2024-06-08 02:22:15,543] Trial 47 pruned. Trial was pruned at epoch 1.
2024-06-08 02:22:15,620 - LSTM model built and compiled
[I 2024-06-08 02:22:25,340] Trial 48 pruned. Trial was pruned at epoch 0.
2024-06-08 02:22:25,579 - LSTM model built and compiled



Epoch 1: val_loss improved from inf to 0.38286, saving model to checkpoint_trial_49.h5

Epoch 2: val_loss improved from 0.38286 to 0.37785, saving model to checkpoint_trial_49.h5

Epoch 3: val_loss did not improve from 0.37785

Epoch 4: val_loss improved from 0.37785 to 0.36667, saving model to checkpoint_trial_49.h5

Epoch 5: val_loss improved from 0.36667 to 0.35334, saving model to checkpoint_trial_49.h5

Epoch 6: val_loss improved from 0.35334 to 0.34825, saving model to checkpoint_trial_49.h5

Epoch 7: val_loss did not improve from 0.34825

Epoch 8: val_loss did not improve from 0.34825

Epoch 9: val_loss did not improve from 0.34825

Epoch 10: val_loss did not improve from 0.34825

Epoch 11: val_loss improved from 0.34825 to 0.34215, saving model to checkpoint_trial_49.h5

Epoch 12: val_loss did not improve from 0.34215

Epoch 13: val_loss did not improve from 0.34215

Epoch 14: val_loss did not improve from 0.34215

Epoch 15: val_loss did not improve from 0.34215

Epoch 16: val

[I 2024-06-08 02:23:16,331] Trial 49 finished with value: 0.3421517312526703 and parameters: {'learning_rate': 0.005217557501351504, 'lstm_units': [128, 128], 'dense_units': [128, 64, 32], 'dropout_rate_lstm': 0.12934319608284747, 'dropout_rate_dense': 0.4159856414903692, 'dropout_rate_cnn': 0.23127158049843335, 'l2_reg': 9.676652152509728e-05, 'cnn_filters': [64, 128]}. Best is trial 42 with value: 0.29762405157089233.


In [4]:

model = build_model(
    learning_rate=best_trial.params['learning_rate'],
    cnn_filters=best_trial.params['cnn_filters'],
    lstm_units=best_trial.params['lstm_units'],
    dense_units=best_trial.params['dense_units'],
    dropout_rate_lstm=best_trial.params['dropout_rate_lstm'],
    dropout_rate_dense=best_trial.params['dropout_rate_dense'],
    dropout_rate_cnn=best_trial.params['dropout_rate_cnn'],
    l2_reg=best_trial.params['l2_reg'],
    sequence_length=SEQ_LENGTH
)

checkpoint_path = f'checkpoint_trial_{best_trial.number}.h5'
if os.path.exists(checkpoint_path):
    model.load_weights(checkpoint_path)

history = model.fit(train_sequences, train_targets, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(val_sequences, val_targets), callbacks=[early_stopping, lr_scheduler])
model.save(MODEL_PATH)


2024-06-08 02:23:16,376 - LSTM model built and compiled


ValueError: Unable to load weights saved in HDF5 format into a subclassed Model which has not created its variables yet. Call the Model first, then load the weights.